# Testing 2QAN on different coupling graphs and larger benchmark circuits

In this notebook, we benchmark the performance of py2qan across a range of circuit types (100 qubits each unless otherwise marked):

- Quantum Approximate Optimization Algorithm (QAOA)
- Quantum volume (QV) calculation
- Quantum Fourier transform (QFT)
- Square Heisenberg model Trotterized Hamiltonian simulation
- Quantum computational neural network (QCNN)
- PREPARE & SELECT on a 25 qubit GHZ state

### Circuit definitions
For the QAOA, QFT, Square Heisenberg, and QV circuits, we copy the corresponding OpenQASM 2 code from qiskit's benchpress library. For the QCNN and Prepare & Select circuits, we generate QASM 2 code using our own implementations in qiskit and cirq, decomposed into the restricted basis set:

In [1]:
folder = "./qasm_circuits/qasm2/"

qasm_files = [folder + file for file in [
    "benchpress/qaoa_barabasi_albert_N100_3reps_basis_rz_rx_ry_cx.qasm",
    "benchpress/qv_N100_12345_basis_rz_rx_ry_cx.qasm",
    "benchpress/qft_N100_basis_rz_rx_ry_cx.qasm",
    "benchpress/square_heisenberg_N100_basis_rz_rx_ry_cx.qasm",
    "ucc/prep_select_N25_ghz_basis_rz_rx_ry_h_cx.qasm",
    "ucc/qcnn_N100_7layers_basis_rz_rx_ry_h_cx.qasm"
    ]]

In [5]:
qasm_strings = []
for filename in qasm_files:
    with open(filename, "r") as file:
        qasm_strings.append(file.read())


We also generate different coupling graphs (in the form of lists of edges) to test circuit mapping and routing.

In [2]:
from benchmarks.scripts.generate_layouts import generate_tilted_square_coupling_list, generate_heavy_hex_coupling_list
coupling_lists = {"cl_sycamore": generate_tilted_square_coupling_list(15, 12), "cl_hhex": generate_heavy_hex_coupling_list(7)} 

In [3]:
import numpy as np
import time
# Import 2QAN compiler passes
from py2qan import BenchArch
from py2qan import HeuristicMapper
from py2qan import QuRouter
# Import qiskit 
import qiskit
from qiskit.transpiler import CouplingMap

Copied `qs_compiler` function from qiskit comparison notebook, removed the code comparing with qiskit for now.

In [4]:
def qs_compiler(qasm, coupling_map, layers=1, trials=1):
    qs_circ = None
    qs_swap = (0, 0) # the number of swaps in the format (#swaps,#swaps merged with circuit gate)
    qs_g2 = 0 # the number of two-qubit gates without decomposition
    # Perform qubit mapping, routing, and scheduling only, without gate decomposition
    for trial in range(trials):
        # Both QAP and Qiskit mappers output inital qubit maps randomly, 
        # one can run the mapper several times to achieve better compilation results
        # Initial qubit mapping 
        hmapper = HeuristicMapper(qasm, coupling_map=coupling_map)
        init_map = hmapper.run_qiskit(max_iterations=5)
        
        # init_map = {circuit qubit index:device qubit index}
        print('The initial qubit map is \n', init_map)
        start = time.time()
        # Routing and scheduling, takes init_map as input
        router = QuRouter(qasm, init_map=init_map, coupling_map=coupling_map)
        # For quantum simulation circuits, we assume each layer has the same time steps
        qs_circ, swaps = router.run(layers=layers, msmt='True')
        # qs_circ0 is the routed circuit without gate decomposition
        # swaps1 is a tuple=(#swaps,#swaps merged with circuit gate)
        end = time.time()
        print("Router run time: ", end - start)
        return qs_circ, swaps


In [6]:
qs_compiler(qasm_strings[0], CouplingMap(coupling_lists["cl_sycamore"]))

The initial qubit map is 
 {0: 12, 1: 43, 2: 56, 3: 84, 4: 90, 5: 100, 6: 74, 7: 89, 8: 81, 9: 92, 10: 48, 11: 91, 12: 32, 13: 31, 14: 28, 15: 64, 16: 63, 17: 4, 18: 0, 19: 1, 20: 37, 21: 35, 22: 20, 23: 21, 24: 5, 25: 7, 26: 19, 27: 98, 28: 2, 29: 71, 30: 16, 31: 6, 32: 65, 33: 47, 34: 66, 35: 97, 36: 26, 37: 44, 38: 55, 39: 80, 40: 8, 41: 49, 42: 102, 43: 61, 44: 99, 45: 36, 46: 53, 47: 95, 48: 3, 49: 13, 50: 23, 51: 72, 52: 85, 53: 54, 54: 45, 55: 29, 56: 70, 57: 96, 58: 86, 59: 50, 60: 82, 61: 76, 62: 22, 63: 46, 64: 25, 65: 51, 66: 68, 67: 11, 68: 27, 69: 79, 70: 83, 71: 18, 72: 78, 73: 40, 74: 75, 75: 62, 76: 42, 77: 10, 78: 58, 79: 94, 80: 24, 81: 38, 82: 60, 83: 57, 84: 101, 85: 41, 86: 52, 87: 67, 88: 9, 89: 14, 90: 69, 91: 59, 92: 103, 93: 73, 94: 87, 95: 77, 96: 15, 97: 93, 98: 17, 99: 88, 100: 30, 101: 33, 102: 34, 103: 39}
Router run time:  5.59675407409668


(<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x11db32d50>, (349, 47))

Router seems to get stuck for these circuits- had to stop after ~5 min without any circuits routed.

In [10]:
for qs in qasm_strings[1:]:
    qs_compiler(qs, CouplingMap(coupling_lists["cl_sycamore"]))

The initial qubit map is 
 {0: 12, 1: 43, 2: 53, 3: 56, 4: 4, 5: 36, 6: 100, 7: 15, 8: 26, 9: 96, 10: 90, 11: 69, 12: 60, 13: 77, 14: 98, 15: 91, 16: 61, 17: 31, 18: 42, 19: 33, 20: 84, 21: 44, 22: 46, 23: 8, 24: 10, 25: 28, 26: 86, 27: 7, 28: 67, 29: 64, 30: 55, 31: 65, 32: 39, 33: 83, 34: 6, 35: 17, 36: 48, 37: 9, 38: 45, 39: 75, 40: 49, 41: 88, 42: 29, 43: 62, 44: 94, 45: 51, 46: 0, 47: 47, 48: 72, 49: 40, 50: 19, 51: 99, 52: 71, 53: 59, 54: 80, 55: 95, 56: 76, 57: 25, 58: 5, 59: 21, 60: 2, 61: 54, 62: 70, 63: 58, 64: 63, 65: 14, 66: 3, 67: 41, 68: 102, 69: 81, 70: 73, 71: 16, 72: 23, 73: 18, 74: 24, 75: 103, 76: 85, 77: 93, 78: 37, 79: 22, 80: 97, 81: 13, 82: 30, 83: 35, 84: 66, 85: 27, 86: 11, 87: 32, 88: 57, 89: 87, 90: 79, 91: 68, 92: 34, 93: 78, 94: 20, 95: 74, 96: 92, 97: 1, 98: 52, 99: 89, 100: 38, 101: 101, 102: 50, 103: 82}


KeyboardInterrupt: 

Router seems to get stuck for this coupling map- had to stop after ~5 min without the circuit being routed.

In [11]:
qs_compiler(qasm_strings[0], CouplingMap(coupling_lists["cl_hhex"]))

The initial qubit map is 
 {0: 0, 1: 1, 2: 2, 3: 6, 4: 42, 5: 43, 6: 47, 7: 48, 8: 49, 9: 72, 10: 73, 11: 75, 12: 109, 13: 110, 14: 114, 15: 88, 16: 26, 17: 22, 18: 99, 19: 61, 20: 15, 21: 87, 22: 21, 23: 57, 24: 38, 25: 39, 26: 70, 27: 91, 28: 106, 29: 67, 30: 85, 31: 71, 32: 29, 33: 84, 34: 54, 35: 9, 36: 50, 37: 107, 38: 28, 39: 80, 40: 14, 41: 8, 42: 98, 43: 19, 44: 90, 45: 27, 46: 12, 47: 16, 48: 36, 49: 20, 50: 60, 51: 105, 52: 96, 53: 35, 54: 112, 55: 51, 56: 52, 57: 17, 58: 45, 59: 40, 60: 53, 61: 56, 62: 104, 63: 32, 64: 86, 65: 11, 66: 79, 67: 13, 68: 10, 69: 41, 70: 64, 71: 103, 72: 77, 73: 108, 74: 4, 75: 37, 76: 83, 77: 65, 78: 66, 79: 34, 80: 76, 81: 69, 82: 100, 83: 7, 84: 18, 85: 46, 86: 44, 87: 113, 88: 97, 89: 82, 90: 78, 91: 102, 92: 5, 93: 33, 94: 74, 95: 55, 96: 3, 97: 81, 98: 111, 99: 89, 100: 92, 101: 23, 102: 93, 103: 24, 104: 94, 105: 25, 106: 95, 107: 30, 108: 31, 109: 101, 110: 58, 111: 59, 112: 62, 113: 63, 114: 68}


KeyboardInterrupt: 